# Library imports and functions

In [4]:
import pandas as pd
import numpy as np
import os
from scipy.stats import kstest # to check if the samples are normally distributed
from scipy.stats import levene #Levene's test
from scipy.stats import ttest_ind # 2 sample t-test 
from scipy.stats import t # confidence Interval of the difference
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from io import StringIO

os.chdir("C:/Users/vinotsek/Desktop/Statistical Analysis")
def convert_to_million_currency(series):
    return series.apply(lambda x: "${:,.2f}M".format(x / 1_000000))

# Read, clean & manipulate the service offer data

In [11]:
# CX denotes Cisco services business
AS=pd.read_excel("raw_data/AS.xlsx")
TS=pd.read_excel("raw_data/TS.xlsx")
AS.drop(columns=['AS Architecture','AS Offer Type','AS Technology'],axis=1,inplace=True)
CX = pd.concat([AS,TS],axis=0)
CX = CX[CX["Fiscal Year"].isin([2024,2023])]
CX["ERP Deal ID"].fillna("Unknown",inplace=True)
CX["Inter"] = [row.split('-')[1] if len(row.split('-')) > 1 else None for row in CX["Sub SCMS"]]
map = CX.groupby(["End Customer Global Ultimate Name","Inter"])["CA Service Bookings Net"].sum().sort_values(ascending = False).reset_index().drop_duplicates(subset=["End Customer Global Ultimate Name"])
map.rename(columns={"Inter":"Tier"},inplace=True)
CX["Tier"] = CX["End Customer Global Ultimate Name"].map(dict(zip(map["End Customer Global Ultimate Name"],map["Tier"])))
CX.drop("Inter",axis=1,inplace=True)
#CX=CX[CX["Tier"].isin(["PREMIER","KEY","MAJOR"])]
#convert_to_million_currency(CX.groupby("CX Product Portfolio")["CA Service Bookings Net"].sum())

#New column to map EC deals
EC_Deals = CX[CX["CX Product Category"] == 'Expert Care Svcs'].groupby("ERP Deal ID")["CA Service Bookings Net"].sum().sort_values(ascending=False).reset_index()
EC_Deals = EC_Deals[(EC_Deals["ERP Deal ID"]!="Unknown")&(EC_Deals["ERP Deal ID"]!="UNKNOWN")]
EC_Deals = EC_Deals[EC_Deals["CA Service Bookings Net"]>=50000]
EC_Deals["EC_Deal"]="Y"
EC_Deals.drop(columns="CA Service Bookings Net",inplace=True)
EC_dict = dict(zip(EC_Deals["ERP Deal ID"],EC_Deals["EC_Deal"]))
CX["EC_Deal"] = CX["ERP Deal ID"].map(EC_dict)
CX["EC_Deal"].fillna("N",inplace=True)

#New column to map ST deals
ST_Deals = CX[CX["CX Product Portfolio"]=="Success Tracks"].groupby("ERP Deal ID")["CA Service Bookings Net"].sum().sort_values(ascending=False).reset_index()
ST_Deals = ST_Deals[(ST_Deals["ERP Deal ID"]!="Unknown")&(ST_Deals["ERP Deal ID"]!="UNKNOWN")]
ST_Deals = ST_Deals[ST_Deals["CA Service Bookings Net"]>=10000]
ST_Deals["ST_Deal"]="Y"
ST_Deals.drop(columns="CA Service Bookings Net",inplace=True)
ST_dict = dict(zip(ST_Deals["ERP Deal ID"],ST_Deals["ST_Deal"]))
CX["ST_Deal"] = CX["ERP Deal ID"].map(ST_dict)
CX["ST_Deal"].fillna("N",inplace=True)

#ST Discount dataset
CX1= CX[CX["CX Product Portfolio"]=="Success Tracks"]
CX1= CX1[CX1["ST_Deal"]=="Y"]
CX1=CX1.groupby(["ERP Deal ID","EC_Deal"]).agg(List_Price = ("CA Service Bookings Standard List","sum"),Net_Price=("CA Service Bookings Net","sum")).reset_index()
CX1["Discount"]=(CX1["List_Price"]-CX1["Net_Price"])/CX1["List_Price"]
#CX1=CX1[(CX1["Discount"]>=0) & (CX1["Discount"]<=1)] #removing outliers

#EC Discount dataset
CX2= CX[CX["CX Product Category"] == 'Expert Care Svcs']
CX2= CX2[CX2["EC_Deal"]=="Y"]
CX2=CX2.groupby(["ERP Deal ID","ST_Deal"]).agg(List_Price = ("CA Service Bookings Standard List","sum"),Net_Price=("CA Service Bookings Net","sum")).reset_index()
CX2["Discount"]=(CX2["List_Price"]-CX2["Net_Price"])/CX2["List_Price"]
#CX2=CX2[(CX2["Discount"]>=0) & (CX2["Discount"]<=1)] #removing outliers

# rewriting the manipulated intermediate dataset back to the folder
with pd.ExcelWriter("EC insertion to ST_Discount Analysis_output.xlsx") as writer:
    CX.to_excel(writer,sheet_name="raw_data",index=False)
    CX1.to_excel(writer,sheet_name="ST",index=False)
    CX2.to_excel(writer,sheet_name="EC",index=False)

C:\Users\vinotsek\jupyterenv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\vinotsek\jupyterenv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\vinotsek\AppData\Local\Temp\ipykernel_27088\3987222081.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operatio

# Statistical test to identify if the distribution is normal and the variance of the sample are equal

In [53]:
# Perform Kolmogorov-Smirnov test on CX1 & CX2
ks_statistic, p_value = kstest(CX2["Discount"], 'norm')

# Print results
print("Kolmogorov-Smirnov Statistic:", ks_statistic)
print("p-value:", p_value)

#The result shows the distribution is not normal as the null hypothesis is rejected. 
#But still we can use parametric test as we have sufficient sample in both the category


# Perform levene test to check if the variance of the sample are equal
# Assuming sample1 and sample2 are your two samples
statistic, p_value = levene(CX1[CX1["EC_Deal"]=="Y"]["Discount"], CX1[CX1["EC_Deal"]=="N"]["Discount"])
print("Levene's Test Statistic:", statistic)
print("p-value:", p_value)
#reject the null hypothesis and conclude that the variance of the sample are not equal

Kolmogorov-Smirnov Statistic: 0.6114219194349028
p-value: 0.0


In [67]:
# Assuming 'sample1' and 'sample2' are your datasets
t_statistic, p_value = ttest_ind(CX1[CX1["EC_Deal"]=="Y"]["Discount"], CX1[CX1["EC_Deal"]=="N"]["Discount"], equal_var=False)  # equal_var=False for unequal variances
print(t_statistic,p_value)

6.1998741318293975 3.701877189721089e-09


# Histograms of the discount distribution 

In [2]:
# Histogram for ST with EC and ST without EC
ST_with_EC = CX1[CX1["EC_Deal"]=="Y"]["Discount"]
ST_without_EC = CX1[CX1["EC_Deal"]=="N"]["Discount"]

# Perform two-sample t-test
t_statistic, p_value = ttest_ind(ST_with_EC, ST_without_EC, equal_var=False)

# Create overlaid histograms to visualize the distribution of the two groups
sns.histplot(ST_with_EC, color='navy', alpha=0.7, label='ST_with_EC', kde=True)
sns.histplot(ST_without_EC, color='lightsteelblue', alpha=0.3, label='ST_without_EC', kde=True)

# Plot mean lines and annotate mean values
mean1 = np.mean(ST_with_EC)
mean2 = np.mean(ST_without_EC)
plt.axvline(mean1, color='navy', linestyle='dashed', linewidth=1, label=f'Average_discount ST_with_EC: {mean1:.2f}')
plt.axvline(mean2, color='lightsteelblue', linestyle='dashed', linewidth=1, label=f'Average_discount ST_without_EC: {mean2:.2f}')

# Add legend
plt.legend()

# Add labels and title
plt.xlabel('Discount')
plt.ylabel('Deal Count')
#plt.title('Distribution of Two Groups')

# Save the plot as an image file
plt.savefig('histogram_with_means(1).png')

plt.show()

print("T-statistic:", t_statistic)
print("P-value:", p_value)
print("mean1:",mean1)
print("mean2:",mean2)

In [3]:
# Histogram for EC with ST and EC without ST 
ST_with_EC = CX1[CX1["EC_Deal"]=="Y"]["Discount"]
ST_without_EC = CX1[CX1["EC_Deal"]=="N"]["Discount"]

EC_with_ST = CX2[CX2["ST_Deal"]=="Y"]["Discount"]
EC_without_ST = CX2[CX2["ST_Deal"]=="N"]["Discount"]

# Perform two-sample t-test
t_statistic, p_value = ttest_ind(EC_with_ST, EC_without_ST, equal_var=False)

# Create overlaid histograms to visualize the distribution of the two groups
sns.histplot(EC_with_ST, color='navy', alpha=0.7, label='EC_with_ST', kde=True)
sns.histplot(EC_without_ST, color='lightsteelblue', alpha=0.3, label='EC_without_ST', kde=True)

# Plot mean lines and annotate mean values
mean1 = np.mean(EC_with_ST)
mean2 = np.mean(EC_without_ST)
plt.axvline(mean1, color='navy', linestyle='dashed', linewidth=1, label=f'Average_discount EC_with_ST: {mean1:.2f}')
plt.axvline(mean2, color='lightsteelblue', linestyle='dashed', linewidth=1, label=f'Average_discount EC_without_ST: {mean2:.2f}')

# Add legend
plt.legend()

# Add labels and title
plt.xlabel('Discount')
plt.ylabel('Deal Count')
#plt.title('Distribution of Two Groups')

# Save the plot as an image file
plt.savefig('histogram_with_means(2).png')

plt.show()

print("T-statistic:", t_statistic)
print("P-value:", p_value)
print("mean1:",mean1)
print("mean2:",mean2)

# Confidence Interval of two sample t-test

In [103]:
# Creating confidence interval of two sample t-test
import numpy as np
from scipy.stats import t

# Assuming sample1 and sample2 are your two samples
sample1 = CX1[CX1["EC_Deal"]=="Y"]["Discount"]
sample2 = CX1[CX1["EC_Deal"]=="N"]["Discount"]

# Calculate the difference in means
mean_difference = np.mean(sample1) - np.mean(sample2)

# Calculate the standard errors
se1 = np.std(sample1, ddof=1) / np.sqrt(len(sample1))
se2 = np.std(sample2, ddof=1) / np.sqrt(len(sample2))

# Calculate the degrees of freedom
degrees_freedom = ((se1 ** 2 / len(sample1)) + (se2 ** 2 / len(sample2))) ** 2 / (((se1 ** 2 / len(sample1)) ** 2) / (len(sample1) - 1) + ((se2 ** 2 / len(sample2)) ** 2) / (len(sample2) - 1))

# Calculate the t-statistic
t_statistic = mean_difference / np.sqrt(se1 ** 2 + se2 ** 2)

# Calculate the critical value
alpha = 0.05  # 95% confidence interval
critical_value = t.ppf(1 - alpha / 2, degrees_freedom)  # two-tailed test

# Calculate the margin of error
margin_of_error = critical_value * np.sqrt(se1 ** 2 + se2 ** 2)

# Calculate the confidence interval
ci_lower = mean_difference - margin_of_error
ci_upper = mean_difference + margin_of_error

# Print confidence interval
print(mean_difference, margin_of_error)
print("Confidence Interval: [{:.4f}, {:.4f}]".format(ci_lower, ci_upper))

0.10716587288774271 0.026562486004159735
Confidence Interval: [0.0806, 0.1337]


In [102]:
# Creating confidence interval of two sample t-test
import numpy as np
from scipy.stats import t

# Assuming sample1 and sample2 are your two samples
sample1 = CX2[CX2["ST_Deal"]=="Y"]["Discount"]
sample2 = CX2[CX2["ST_Deal"]=="N"]["Discount"]

# Calculate the difference in means
mean_difference = np.mean(sample1) - np.mean(sample2)

# Calculate the standard errors
se1 = np.std(sample1, ddof=1) / np.sqrt(len(sample1))
se2 = np.std(sample2, ddof=1) / np.sqrt(len(sample2))

# Calculate the degrees of freedom
degrees_freedom = ((se1 ** 2 / len(sample1)) + (se2 ** 2 / len(sample2))) ** 2 / (((se1 ** 2 / len(sample1)) ** 2) / (len(sample1) - 1) + ((se2 ** 2 / len(sample2)) ** 2) / (len(sample2) - 1))

# Calculate the t-statistic
t_statistic = mean_difference / np.sqrt(se1 ** 2 + se2 ** 2)

# Calculate the critical value
alpha = 0.05  # 95% confidence interval
critical_value = t.ppf(1 - alpha / 2, degrees_freedom)  # two-tailed test

# Calculate the margin of error
margin_of_error = critical_value * np.sqrt(se1 ** 2 + se2 ** 2)

# Calculate the confidence interval
ci_lower = mean_difference - margin_of_error
ci_upper = mean_difference + margin_of_error

# Print confidence interval
print(mean_difference, margin_of_error)
print("Confidence Interval: [{:.4f}, {:.4f}]".format(ci_lower, ci_upper))

-0.028480855078398926 0.018699082846744527
Confidence Interval: [-0.0472, -0.0098]


# Pyramid slide excel output (Summarizes the annual order value based on size categories and offers)

In [99]:
# PKM and customer count

# CX_= pd.read_excel("raw_data/CX AOV.xlsx")
# CX_ =CX_[~CX_["CAV End Customer BU ID"].isin([-999])]
# CX_["Inter"] = [row.split('-')[1] if len(row.split('-')) > 1 else None for row in CX_["Sub SCMS"]]
# map = CX_.groupby(["CAV End Customer BU ID","Inter"])["AOV"].sum().sort_values(ascending = False).reset_index().drop_duplicates(subset=["CAV End Customer BU ID"])
# map.rename(columns={"Inter":"Tier"},inplace=True)
# CX_["Tier"] = CX_["CAV End Customer BU ID"].map(dict(zip(map["CAV End Customer BU ID"],map["Tier"])))
# CX_.drop("Inter",axis=1,inplace=True)

AOV_Category = CX_[["CAV End Customer BU ID","Tier"]].drop_duplicates()
AOV_Category = AOV_Category.rename(columns={"Tier":"Category"})

category_mapping = {
    'PREMIER': 'PREMIER',   
    'KEY': 'KEY',   
    'MAJOR': 'MAJOR',  
    'SELECT': 'SELECT',  
    'MIDSIZE': 'OTHER',  
    'SMALL': 'OTHER'   
}

# Apply the mapping to the 'Category' column
AOV_Category['Category'] = AOV_Category['Category'].replace(category_mapping)
AOV_Category["Category"].fillna("OTHER",inplace=True)

#CX
CX_AOV = CX_.groupby('CAV End Customer BU ID')['AOV'].agg(CX_AOV='sum').reset_index()
Overall_CX_AOV = CX_AOV['CX_AOV'].sum()

#ST
ST_AOV = CX_[CX_["CX Product Category"].isin(["Level 1","Level 2","Level Unknown"])]
ST_AOV = ST_AOV.groupby('CAV End Customer BU ID')['AOV'].agg(ST_AOV='sum').reset_index()
Overall_ST_AOV = ST_AOV['ST_AOV'].sum()

#SSPT
SSPT_AOV = CX_[CX_["CX Product Category"]=="Solution Support"]
SSPT_AOV = SSPT_AOV.groupby('CAV End Customer BU ID')['AOV'].agg(SSPT_AOV='sum').reset_index()
Overall_SSPT_AOV = SSPT_AOV['SSPT_AOV'].sum()

#EC
EC_AOV = CX_[CX_["CX Product Category"]=="Expert Care Svcs"]
EC_AOV = EC_AOV.groupby('CAV End Customer BU ID')['AOV'].agg(EC_AOV='sum').reset_index()
Overall_EC_AOV = EC_AOV['EC_AOV'].sum()

table_list=[AOV_Category,CX_AOV,ST_AOV,SSPT_AOV,EC_AOV]

base_df= CX_["CAV End Customer BU ID"]

base_df=pd.DataFrame(base_df.drop_duplicates())

# Iterate through the list and left join each table to the base_df
for table in table_list:
    base_df = base_df.merge(table, on='CAV End Customer BU ID', how='left')
    
base_df = base_df.fillna(0)

base_df["#CX"] = np.where(base_df["CX_AOV"]>10000,1,0)
base_df["#ST"] = np.where(base_df["ST_AOV"]>10000,1,0)
base_df["#SSPT"] = np.where(base_df["SSPT_AOV"]>10000,1,0)
base_df["#EC"] = np.where(base_df["EC_AOV"]>50000,1,0)

base_df.to_excel("Pyramid_raw1.xlsx",index=False)

C:\Users\vinotsek\AppData\Local\Temp\ipykernel_51408\3319432314.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AOV_Category["Category"].fillna("OTHER",inplace=True)


In [89]:
# AOV category & size

CX_= pd.read_excel("raw_data/CX AOV.xlsx")
CX_ =CX_[~CX_["CAV End Customer BU ID"].isin([-999])]
CX_["Inter"] = [row.split('-')[1] if len(row.split('-')) > 1 else None for row in CX_["Sub SCMS"]]
map = CX_.groupby(["CAV End Customer BU ID","Inter"])["AOV"].sum().sort_values(ascending = False).reset_index().drop_duplicates(subset=["CAV End Customer BU ID"])
map.rename(columns={"Inter":"Tier"},inplace=True)
CX_["Tier"] = CX_["CAV End Customer BU ID"].map(dict(zip(map["CAV End Customer BU ID"],map["Tier"])))
CX_.drop("Inter",axis=1,inplace=True)
CX_=CX_[CX_["Tier"].isin(["PREMIER","KEY","MAJOR"])]

AOV_Category = CX_.groupby(["CAV End Customer BU ID")["AOV"].sum().reset_index()
pd.Series(CX["AOV_Customer"].unique()).describe().apply(lambda x: f"{x:,.0f}")
conditions = [4r354 
    AOV_Category['AOV'] < 10000,
    (AOV_Category['AOV'] >= 10000) & (AOV_Category['AOV'] < 500000),
    (AOV_Category['AOV'] >= 500000) & (AOV_Category['AOV'] < 1000000),
    (AOV_Category['AOV'] >= 1000000) & (AOV_Category['AOV'] < 5000000),
    (AOV_Category['AOV'] >= 5000000) & (AOV_Category['AOV'] < 10000000),
    (AOV_Category['AOV'] >= 10000000) & (AOV_Category['AOV'] < 25000000),
    (AOV_Category['AOV'] >= 25000000)
]
values = ['<$10K','$10K - $500K', '$500K - $1M', '$1M - $5M','$5M- $10M','$10M- $25M','>$25M']
AOV_Category['category'] = np.select(conditions, values, default='')
AOV_Category=AOV_Category.iloc[:,[0,2]]

AOV_Category = CX_[["CAV End Customer BU ID","Tier"]]
AOV_Category.rename(columns={"Tier":"Category"})

#CX
CX_AOV = CX_.groupby('CAV End Customer BU ID')['AOV'].agg(CX_AOV='sum').reset_index()
Overall_CX_AOV = CX_AOV['CX_AOV'].sum()

#ST
ST_AOV = CX_[CX_["CX Product Category"].isin(["Level 1","Level 2","Level Unknown"])]
ST_AOV = ST_AOV.groupby('CAV End Customer BU ID')['AOV'].agg(ST_AOV='sum').reset_index()
Overall_ST_AOV = ST_AOV['ST_AOV'].sum()

#SSPT
SSPT_AOV = CX_[CX_["CX Product Category"]=="Solution Support"]
SSPT_AOV = SSPT_AOV.groupby('CAV End Customer BU ID')['AOV'].agg(SSPT_AOV='sum').reset_index()
Overall_SSPT_AOV = SSPT_AOV['SSPT_AOV'].sum()

#EC
EC_AOV = CX_[CX_["CX Product Category"]=="Expert Care Svcs"]
EC_AOV = EC_AOV.groupby('CAV End Customer BU ID')['AOV'].agg(EC_AOV='sum').reset_index()
Overall_EC_AOV = EC_AOV['EC_AOV'].sum()

table_list=[AOV_Category,CX_AOV,ST_AOV,SSPT_AOV,EC_AOV]

base_df= CX_["CAV End Customer BU ID"]

base_df=pd.DataFrame(base_df.drop_duplicates())

# Iterate through the list and left join each table to the base_df
for table in table_list:
    base_df = base_df.merge(table, on='CAV End Customer BU ID', how='left')
    
base_df = base_df.fillna(0)

base_df["category"]=base_df["category"].replace(0,"<$10K")

base_df["#CX"] = np.where(base_df["CX_AOV"]>10000,1,0)
base_df["#ST"] = np.where(base_df["ST_AOV"]>10000,1,0)
base_df["#SSPT"] = np.where(base_df["SSPT_AOV"]>10000,1,0)
base_df["#EC"] = np.where(base_df["EC_AOV"]>10000,1,0)

#base_df.to_excel("Pyramid_raw2.xlsx",index=False)

# CSAT Data manipulation on Customer Satisfaction Score

In [67]:
# AOV = pd.read_excel("raw_data/CX AOV.xlsx")
CSAT = pd.read_csv("raw_data\CSAT.csv")
CSAT=CSAT[CSAT["C_FISCALYR"].isin(["FY2024","FY2023"])]
ST_AOV = AOV[AOV["CX Product Portfolio"]=="Success Tracks"].groupby("CAV End Customer BU ID")["AOV"].sum().sort_values(ascending=False).reset_index()
ST_AOV = ST_AOV[ST_AOV["AOV"]>=10000]
AOV1 = AOV[AOV["CAV End Customer BU ID"].isin(ST_AOV["CAV End Customer BU ID"])]
EC_AOV = AOV1[AOV1["CX Product Category"]=="Expert Care Svcs"].groupby("CAV End Customer BU ID")["AOV"].sum().reset_index()
EC_AOV = EC_AOV[EC_AOV["AOV"]>=10000]
EC_AOV["with_or_without_EC"] = "with_EC"
ST_AOV["with_or_without_EC"]=ST_AOV["CAV End Customer BU ID"].map(dict(zip(EC_AOV["CAV End Customer BU ID"],EC_AOV["with_or_without_EC"])))
ST_AOV["with_or_without_EC"].fillna("without_EC",inplace=True)
CSAT=CSAT[CSAT["E_CAVCUSTOMERBUID"].isin(ST_AOV["CAV End Customer BU ID"])]
mapping_dict = dict(zip(ST_AOV["CAV End Customer BU ID"], ST_AOV["with_or_without_EC"]))
CSAT["with_or_without_EC"] = CSAT["E_CAVCUSTOMERBUID"].map(mapping_dict)
CSAT.to_excel("CSAT_Output.xlsx")

C:\Users\vinotsek\AppData\Local\Temp\ipykernel_51408\3379410815.py:2: DtypeWarning: Columns (5,65,131) have mixed types. Specify dtype option on import or set low_memory=False.
  CSAT = pd.read_csv("raw_data\CSAT.csv")
C:\Users\vinotsek\AppData\Local\Temp\ipykernel_51408\3379410815.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ST_AOV["with_or_without_EC"].fillna("without_EC",inplace=True)


# New Logo (Identify the new customers for a given offer within the complementary offer)

In [7]:
# New Logo for EC 
ST = CX_[CX_["CX Product Portfolio"]=="Success Tracks"]
ST1 = ST.groupby(["CAV End Customer BU ID","Sales Level 2"])["AOV"].sum().sort_values(ascending = False).reset_index()
ST1= ST1[ST1["AOV"]>=161000]
ST1["CAV End Customer BU ID"]=ST1["CAV End Customer BU ID"].astype("str")
ST1["CustSL2"]=ST1["CAV End Customer BU ID"]+"*" +ST1["Sales Level 2"]
ST1=ST1.iloc[:,3]
EC = CX_[CX_["CX Product Category"]=="Expert Care Svcs"]
EC1 = EC.groupby(["CAV End Customer BU ID","Sales Level 2"])["AOV"].sum().sort_values(ascending = False).reset_index()
EC1= EC1[EC1["AOV"]>=50000]
EC1["CAV End Customer BU ID"]=EC1["CAV End Customer BU ID"].astype("str")
EC1["CustSL2"]=EC1["CAV End Customer BU ID"]+"*" +EC1["Sales Level 2"]
EC1=EC1.iloc[:,3]
ST2=pd.DataFrame(ST1[~ST1.isin(EC1)])

#Old Logo of EC for SL2 Avg AOV Mapping
ST["CAV End Customer BU ID"]=ST["CAV End Customer BU ID"].astype("str")
ST["CustSL2"] = ST["CAV End Customer BU ID"]+"*"+ST["Sales Level 2"]
ST3 = ST[ST["CustSL2"].isin(ST1)]
ST3 = ST3[ST3["CustSL2"].isin(EC1)]
ST3 = ST3[["CustSL2"]]
EC["CAV End Customer BU ID"]=EC["CAV End Customer BU ID"].astype("str")
EC["CustSL2"] = EC["CAV End Customer BU ID"]+"*"+EC["Sales Level 2"]
EC2= EC[EC["CustSL2"].isin(ST3["CustSL2"])]
EC2=EC2.groupby(["CAV End Customer BU ID","Sales Level 2"])["AOV"].sum().sort_values(ascending = False).reset_index()
EC2 = EC2.groupby(["Sales Level 2"])["AOV"].mean().sort_values(ascending = False).reset_index()
EC2 = dict(zip(EC2["Sales Level 2"],EC2["AOV"]))

#Mapping (Tier, CAV BU Name, L2 Oppty)
ST2[['CAV End Customer BU ID', 'Sales Level 2']] = ST2['CustSL2'].str.split('*', expand=True)
ST2["New_Logo_Oppty"]=ST2["Sales Level 2"].map(EC2)
map=CX_[["CAV End Customer BU ID","Tier"]].drop_duplicates()
map["CAV End Customer BU ID"] = map["CAV End Customer BU ID"].astype("str")
ST2["Tier"] = ST2["CAV End Customer BU ID"].map(dict(zip(map["CAV End Customer BU ID"],map["Tier"])))
map1=CX_[["CAV End Customer BU ID","CAV End Customer BU Name"]].drop_duplicates()
map1["CAV End Customer BU ID"] = map1["CAV End Customer BU ID"].astype("str")
ST2["CAV End Customer BU Name"] = ST2["CAV End Customer BU ID"].map(dict(zip(map1["CAV End Customer BU ID"],map1["CAV End Customer BU Name"])))
x=ST2["New_Logo_Oppty"].sum()
print("${:,.2f}M".format(x / 1_000000))
print(ST2.shape[0])
ST2.to_excel("EC_New_Logo.xlsx",index=False)

C:\Users\vinotsek\AppData\Local\Temp\ipykernel_27088\2831805099.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ST["CAV End Customer BU ID"]=ST["CAV End Customer BU ID"].astype("str")
C:\Users\vinotsek\AppData\Local\Temp\ipykernel_27088\2831805099.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ST["CustSL2"] = ST["CAV End Customer BU ID"]+"*"+ST["Sales Level 2"]
C:\Users\vinotsek\AppData\Local\Temp\ipykernel_27088\2831805099.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

$207.93M
620


# Printing Commentary

In [237]:
# Redirect stdout to a StringIO object
stdout_backup = sys.stdout
sys.stdout = StringIO()

# Your print statements
print(f"${round(CX['CA Service Bookings Net'].sum()/1e6)}M in ST Bookings (Total) from {CX['ERP Deal ID'].nunique()} ST Deals in last two years.")
print(f"${round(CX[CX['EC_Deal']=='Y']['CA Service Bookings Net'].sum()/1e6)}M in ST Bookings (Total) from {CX[CX['EC_Deal']=='Y']['ERP Deal ID'].nunique()} ST Deals with EC in last two years.")
print("With t-statistic of 7.61 and p value of <0.05, two-sample t-test rejects the null hypothesis that average discounts between ST Deals with EC and ST Deals without EC are equal, at a 95% confidence level")
print("Statistically, the average discount difference between ST deals with EC and those without ranges from 7.9 to 13.4 percentage points, at a 95% confidence level")
#Discounted calculated at aggregated level not average at deal level
print("The potential savings from selling ST deals with EC at the same discount rate as ST deals without EC amount to $",round(CX[CX["EC_Deal"]=="Y"]["CA Service Bookings Standard List"].sum()*(0.685-0.628)/1e6,0),"M over the last two years.")

# Get the output as a string
output_string = sys.stdout.getvalue()

# Split the output string into a list based on newline character
output_list = output_string.strip().split('\n')

# Restore stdout
sys.stdout = stdout_backup

output_df = pd.DataFrame(output_list, columns=['Commentary'])

# Writing to output to excel

In [60]:
with pd.ExcelWriter('latest.xlsx') as writer:
    CX1.to_excel(writer, sheet_name='ST_Bookings',index=False)
    CX2.to_excel(writer, sheet_name='EC_Bookings',index=False)
    CX.to_excel(writer,sheet_name='raw_data',index=False)